In [1]:
def get_formatted_input(messages, context):
    system = "System: This is a chat between a user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions based on the context. The assistant should also indicate when the answer cannot be found in the context."
    instruction = "Please give a full and complete answer for the question."

    for item in messages:
        if item['role'] == "user":
            ## only apply this instruction for the first user turn
            item['content'] = instruction + " " + item['content']
            break

    conversation = '\n\n'.join(["User: " + item["content"] if item["role"] == "user" else "Assistant: " + item["content"] for item in messages]) + "\n\nAssistant:"
    formatted_input = system + "\n\n" + context + "\n\n" + conversation
    
    return formatted_input

In [14]:
# read in the doc
with open("sample_doc.txt", 'r', encoding='utf-8') as file:
    content = file.read()
    
print(content)

Pittsburgh Restaurant Week
Winter: Jan. 15-21, 2024
Summer: Aug. 12-18, 2024

Pittsburgh Restaurant Week in January and August each year features more than 50 restaurants offering special dining deals. During the week-long, city-wide event, dine at one of Pittsburgh's finest restaurants and take advantage of three-course, fixed-price meals specials.

Fish Fry Season
Feb. 14-March 28, 2024

Another Pittsburgh food staple is the Lenten Fish Fry. Beginning on Ash Wednesday and continuing every Friday until Easter, you can find great fish fries held in churches, fire halls and community buildings all over the Pittsburgh area.

South Side Soup Contest
Feb. 24, 2024 | Brashear Association

Soup's on at the South Side Soup Contest. With Butternut Bourbon Bisque and Chipotle Chicken Chorizo, it's mmmmm good! This annual contest warms the heart - and belly. The annual February event allows shoppers in Pittsburgh's funky South Side neighborhood to sample soups from 20 local restaurants for one l

In [20]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "nvidia/Llama3-ChatQA-1.5-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

messages = [
    {"role": "user", "content": "Propose 10 questions that can be answered using the following text, and provide the answers. \n\nExamples: Q1: Where is the Little Italy Days held in 2024? A1: Bloomfield. \n\nQ2: How many kinds of Mac&cheese will be served at Mac & Cheese Festival? A2: More than 30.\n\n"},
]

formatted_input = get_formatted_input(messages, content)
tokenized_prompt = tokenizer(tokenizer.bos_token + formatted_input, return_tensors="pt").to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(input_ids=tokenized_prompt.input_ids, attention_mask=tokenized_prompt.attention_mask, max_new_tokens=256, eos_token_id=terminators)

response = outputs[0][tokenized_prompt.input_ids.shape[-1]:]
summarized_content = tokenizer.decode(response, skip_special_tokens=True)
print(summarized_content)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.22s/it]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 Here are 10 questions and answers that can be answered using the text:

Q3: When is the South Side Soup Contest in 2024? A3: February 24, 2024.

Q4: What is the name of the food festival that celebrates pickling? A4: Picklesburgh.

Q5: When is the Pittsburgh Irish Festival in 2024? A5: September 6-8, 2024.

Q6: What is the name of the food festival that celebrates tacos? A6: The Original Pittsburgh Taco Festival.

Q7: When is the Lawrenceville Cookie Tour in 2024? A7: To be determined.

Q8: When is the Pittsburgh Vegan Expo & Festival in 2024? A8: August 10, 2024.

Q9: When is the Pittsburgh VegFest in 2024? A9: September 29, 2024.

Q10: When is the Big Nosh Jewish Food Festival in 2024? A10: April 7-9, 2024.
